<a href="https://colab.research.google.com/github/pragtim/QuoteGame/blob/main/QuoteGame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Scraping a site to get quotes and author's details.


In [9]:
import requests
from bs4 import BeautifulSoup
from time import sleep
from csv import DictWriter

BASE_URL = "http://quotes.toscrape.com"

def scrape_quotes():
    all_quotes = []
    url = "/page/1"
    while url:
        res = requests.get(f"{BASE_URL}{url}")
        print(f"Now scraping {BASE_URL}{url}")
        soup = BeautifulSoup(res.text, "html.parser")
        quotes = soup.find_all(class_="quote")

        for quote in quotes:
            all_quotes.append({
                "text": quote.find(class_="text").get_text(),
                "author": quote.find(class_="author").get_text(),
                "bio-link": quote.find("a")["href"] 
            })

        next_btn = soup.find(class_="next")
        url = next_btn.find("a")["href"] if next_btn else None
        
    return all_quotes

In [10]:
#creating a csv
def write_quotes(quotes):
    with open("quotes.csv", 'w') as file:
        headers = ["text", "author", "bio-link"]
        csv_writer = DictWriter(file, fieldnames=headers)
        csv_writer.writeheader()
        for quote in quotes:
            csv_writer.writerow(quote)


In [11]:
quotes = scrape_quotes()
write_quotes(quotes)

Now scraping http://quotes.toscrape.com/page/1
Now scraping http://quotes.toscrape.com/page/2/
Now scraping http://quotes.toscrape.com/page/3/
Now scraping http://quotes.toscrape.com/page/4/
Now scraping http://quotes.toscrape.com/page/5/
Now scraping http://quotes.toscrape.com/page/6/
Now scraping http://quotes.toscrape.com/page/7/
Now scraping http://quotes.toscrape.com/page/8/
Now scraping http://quotes.toscrape.com/page/9/
Now scraping http://quotes.toscrape.com/page/10/


In [12]:
from csv import DictReader
from random import choice

#Reading the csv
def read_quotes(filename):
    with open(filename) as file:
        csv_reader = DictReader(file)
        return list(csv_reader)

In [21]:
def start_game(quotes):
    quote = choice(quotes)
    print("The quote is :")
    print(quote["text"])
    print(quote["author"])

    remaining_guesses = 3
    guess = ''
    while guess.lower() != quote["author"].lower() and remaining_guesses:
        guess = input(f"Who said this quote? Guess/es remaining: {remaining_guesses}: ")
        if guess.lower() == quote["author"].lower():
            print("You got it right!")
            break

        remaining_guesses -= 1
        if remaining_guesses == 2:
            res = requests.get(f"{BASE_URL}{quote['bio-link']}")
            soup = BeautifulSoup(res.text, "html.parser")
            birth_date = soup.find(class_="author-born-date").get_text()
            birth_place = soup.find(class_="author-born-location").get_text()
            print(f"Here's hint: The author was born on {birth_date} {birth_place}")
        elif remaining_guesses == 1:
            print(f"Here's a hint. The author's first name starts with {quote['author'][0]}")
            last_initial = quote["author"].split(" ")[1][0]
            print(f"And last name starts with {last_initial}")
        else:
            print(f"Sorry no more guesses remaining. The answer was {quote['author']}")

    again = ''
    while again.lower() not in ('y', 'n'):
        again = input("Would you like to play again (y/n)? ")
    if again.lower() == 'y':
        return start_game(quotes)
    else:
        print("GAME FINISH")



In [22]:

lquotes = read_quotes("quotes.csv")
start_game(quotes)

The quote is :
“Life is what happens to us while we are making other plans.”
Allen Saunders
Who said this quote? Guess/es remaining: 3: mk
Here's hint: The author was born on April 24, 1899 in The United States
Who said this quote? Guess/es remaining: 2: mk
Here's a hint. The author's first name starts with A
And last name starts with S
Who said this quote? Guess/es remaining: 1: nj
Sorry no more guesses remaining. The answer was Allen Saunders
Would you like to play again (y/n)? n
GAME FINISH


In [ ]:
from google.colab import drive
drive.mount('/content/drive')